<a href="https://colab.research.google.com/github/YashShekhar/Generative-AI/blob/main/Build_a_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Chatbot

In [29]:
!pip install -U langchain-groq

In [30]:
!pip install langchain-core langgraph>0.2.27

In [3]:
## Setup API KEY - Google Colab
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [4]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="openai/gpt-oss-120b",
    model_provider="groq"
)

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi! I am Yash")])

AIMessage(content='Hello Yash! 👋 Nice to meet you. How can I help you today?', additional_kwargs={'reasoning_content': 'The user says "Hi! I am Yash". Likely a greeting, maybe wants to respond. Should respond friendly. No special instructions.'}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 77, 'total_tokens': 134, 'completion_time': 0.115624988, 'prompt_time': 0.003754858, 'queue_time': 0.216534622, 'total_time': 0.119379846}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_213abb2467', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6996cecf-f420-49f4-a955-e7d6a6a71c3f-0', usage_metadata={'input_tokens': 77, 'output_tokens': 57, 'total_tokens': 134})

In [11]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content='I’m not able to see your name. If you’d like me to address you personally, just let me know what you’d prefer to be called!', additional_kwargs={'reasoning_content': 'The user asks "What\'s my name?" The assistant doesn\'t have any prior info about the user. According to policy, we must respond that we don\'t know. We can ask politely for clarification.'}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 75, 'total_tokens': 154, 'completion_time': 0.156947468, 'prompt_time': 0.003092402, 'queue_time': 0.212843506, 'total_time': 0.16003987}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8db49de948', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--5f2c8406-fef1-4eaf-be11-272d05c5fc10-0', usage_metadata={'input_tokens': 75, 'output_tokens': 79, 'total_tokens': 154})

In [39]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I am Yash"),
        AIMessage(content='Hello Yash! 👋 How can I help you today?'),
        HumanMessage(content="What's my name?")
    ]
)

AIMessage(content='Your name is Yash.', additional_kwargs={'reasoning_content': 'The user asks "What\'s my name?" They previously said "Hi! I am Yash". So answer: Yash. Should respond accordingly.'}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 104, 'total_tokens': 149, 'completion_time': 0.090939996, 'prompt_time': 0.005472156, 'queue_time': 0.568958771, 'total_time': 0.096412152}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_a09bde29de', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2e138b43-c537-44a0-bbae-50c3b91aaa13-0', usage_metadata={'input_tokens': 104, 'output_tokens': 45, 'total_tokens': 149})

## Message Persistence

In [6]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [7]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
  response = model.invoke(state["messages"])
  return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [8]:
config = {"configurable": {"thread_id": "abc123"}}

In [10]:
query = "Hi! I'm Yash."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output

{'messages': [HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='2ef7dfe9-8a1c-4f77-86f6-ba1dc62078b6'),
  AIMessage(content='I don’t have any information about your name. If you’d like, feel free to tell me what you’d like to be called, and I’ll use that!', additional_kwargs={'reasoning_content': 'The user asks "What\'s my name?" There\'s no prior context. According to policy, we must not fabricate personal info. We should respond that we don\'t have that info. Apologize and ask if they\'d like to share.'}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 75, 'total_tokens': 164, 'completion_time': 0.188806465, 'prompt_time': 0.003646782, 'queue_time': 0.53532412, 'total_time': 0.192453247}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8db49de948', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--27406027-b41e-482e-abd2-0e4015066401-0', usage_metadata={'input_tokens': 75

In [12]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Nice to meet you, Yash! How can I help you today?


In [11]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Yash.


In [12]:
# If we change the config i.e., thread_id. It will start a new conversation

config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don’t have any information about your name. If you’d like to share it, feel free to let me know!


In [13]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Yash.


In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [16]:
prompt_template = ChatPromptTemplate.from_messages([
    (
        "system",
        "You talk like a pirate. Answer all questions to the best of your ability."
    ),
    MessagesPlaceholder(variable_name="messages")
])
prompt_template

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [24]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [18]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
  prompt = prompt_template.invoke(state)
  response = model.invoke(prompt)
  return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [19]:
config = {"configurable": {"thread_id": "abc345"}}

In [20]:
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy, Jim! 'Tis a fine day on the digital seas, be it? How can this salty seadog be of service t' ye? Arr!


In [21]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yer name be Jim, matey! Arr!


In [22]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [32]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
  messages: Annotated[Sequence[BaseMessage], add_messages]
  language: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
  prompt = prompt_template.invoke(state)
  response = model.invoke(prompt)
  return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

momory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [33]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bob! ¿En qué puedo ayudarte hoy?


In [34]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob.


In [35]:
from langchain_core.messages import SystemMessage, trim_messages

In [37]:
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)
trimmer

RunnableLambda(...)

In [40]:
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [45]:
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [48]:
workflow = StateGraph(state_schema=State)

def call_model(state: State):
  print(f"Messages before trimming: {len(state['messages'])}")
  trimmed_messages = trimmer.invoke(state["messages"])
  print(f"Messages after trimming: {len(trimmed_messages)}")
  print("Remaining messages:")
  for msg in trimmed_messages:
        print(f"  {type(msg).__name__}: {msg.content}")
  prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
  response = model.invoke(prompt)
  return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

momory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [49]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

Messages before trimming: 14
Messages after trimming: 12
Remaining messages:
  SystemMessage: you're a good assistant
  HumanMessage: I like vanilla ice cream
  AIMessage: nice
  HumanMessage: whats 2 + 2
  AIMessage: 4
  HumanMessage: thanks
  AIMessage: no problem!
  HumanMessage: having fun?
  AIMessage: yes!
  HumanMessage: What is my name?
  AIMessage: Your name is Bob.
  HumanMessage: What is my name?
================================== Ai Message ==================================

I don’t actually know your name. If you’d like to share it, feel free to let me know!


In [50]:
config = {"configurable": {"thread_id": "abc678"}}

query = "What math problem was asked?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

Messages before trimming: 12
Messages after trimming: 12
Remaining messages:
  SystemMessage: you're a good assistant
  HumanMessage: hi! I'm bob
  AIMessage: hi!
  HumanMessage: I like vanilla ice cream
  AIMessage: nice
  HumanMessage: whats 2 + 2
  AIMessage: 4
  HumanMessage: thanks
  AIMessage: no problem!
  HumanMessage: having fun?
  AIMessage: yes!
  HumanMessage: What math problem was asked?
================================== Ai Message ==================================

The math problem you asked earlier was **“What is 2 + 2?”**.


# Streaming

In [54]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

Messages before trimming: 7
Messages after trimming: 1
Remaining messages:
  HumanMessage: Hi I'm Todd, please tell me a joke.
||||||||||||||||||Sure| thing|,| Todd|!| Here|’s| one| for| you|:

|**|Why| don|’t| scientists| trust| atoms| anymore|?|**

|Because| they| *|make| up|*| everything|!| 😄||